<a href="https://colab.research.google.com/github/jinseriouspark/pricing/blob/main/%5Bdynamic_pricing%5D_04_multi_armed_bandit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic pricing - 04. Multi Armed Bandit


- 의미 : dynamic pricing 에서는 멀티 암드 밴딧 게임을 사용해서 상품이나 서비스의 가격을 정할때, 어떤 가격이 가장 잘 팔리는지 알아보려고 한다. 여러가지 가격을 시도해보면서, 어떤 가격이 사람들에게 가장 합리적인지 알아내고, 해당 가격을 적용하고자 함


- 방법
  1. 탐험 exploration : 각각의 가격대를 몇번씩 시도하며 유저들의 반응을 살펴봄 (각각의 매출 분포를 이해)
  2. 활용 Exploitation : 어느정도 탐험을 하고난 다음, 유저들의 반응이 가장 높았던 조건의 가격대를 선정하고자 함

- 적용:  여러가지 가격을 시도하여 각 가격에서의 판매량(보상)을 관찰하고, 최적의 가격을 찾아내는 데 사용

##  1. Epsilon-greedy algorithm
- 미래를 생각하지 않고 각 단계에서 가장 최선의 선택을 하는 기법. 각 단계에서 최선의 선택을 한 것이 전체적으로도 최선이라는 것을 의미함

- e 의 확률로 탐색을 하고, 1-e 의 확률로 현재까지 가장 좋은 선택을 활용함

In [39]:
import numpy as np


class EpsilonGreedyBandit:
  def __init__(self, k, epsilon, initial_value =0):
    self.k = k # 선택지의 개수
    self.epsilon = epsilon
    self.q_values = np.full(k, initial_value, dtype=float) # 각 선택의 예상 보상
    self.action_counts = np.zeros(k, dtype = int) # 각 선택지의 선택 횟수

  def select_action(self):
    if np.random.rand() < self.epsilon:
      return np.random.randint(self.k) # 탐색 : epsilon보다 작으면 탐색
    else:
      return np.argmax(self.q_values) # 활용 : 예상 보상이 가장 높은 선택지 채택

  def update(self, chosen_action, reward):
    self.action_counts[chosen_action] += 1 # 선택지 값 업데이트
    n = self.action_counts[chosen_action] # 해당 선택지의 action수
    value = self.q_values[chosen_action] # 해당 시점의 보상

    # 새로운 보상 값 계산
    # chosen_action 의 값의 대부분은 기존 보상을, 1 / n 의 비율로 새로운 reward를 반영
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.q_values[chosen_action] = new_value

In [42]:
# 예제 : 선택가능한 것 5개, 가장 좋은 액션 선택할 확률 (1 - eplion)
k = 5
epsilon = 0.1
bandit = EpsilonGreedyBandit(k, epsilon)

# 시뮬레이션
n_simulations = 1000
true_rewards = [0.1, 0.5, 0.8, 0.4, 0.9] # 실제 각 액션의 보상
rewards = np.zeros(n_simulations)

#
for i in range(n_simulations):
    chosen_action = bandit.select_action()
    reward = np.random.rand() < true_rewards[chosen_action]
    bandit.update(chosen_action, reward)
    rewards[i] = reward

In [37]:
print('각 액션의 예상 보상', bandit.q_values)
print('총 예상 보상', np.sum(rewards))

각 액션의 예상 보상 [0.22727273 0.41666667 0.8046875  0.30769231 0.89393939]
총 예상 보상 824.0


## 2. Upper Confidence Bound, UCB
- 각 액션의 평균 보상과 탐색 요소를 결합하여 신뢰구간이 가장 높은 선택지를 선택함
- 아직 충분히 탐색되지 않은 액션을 더 자주 선택하도록 유도

In [52]:
import numpy as np

class UCBBandit:
  def __init__(self, n_actions):
    self.n_actions = n_actions
    self.action_counts = np.zeros(n_actions) # 각 액션의 선택 횟수
    self.q_values = np.zeros(n_actions) # 각 액션의 예상 보상
    self.total_counts = 0 # 액션 선택 횟수

  def select_action(self): # 5가지 액션 중 한가지를 선택함
    if 0 in self.action_counts: # 아직 한번도 선택되지 않은 팔이 있다면 우선 선택
      return np.argmin(self.action_counts)
    else: # UCB값 계산
      # q_values 는 현재까지의 평균 보상
      # 탐색과 탐험의 균형을 맞추기 위해, 각 액션의 보상에 대한 신뢰구간을 계산
      # 평균 보상 + np.sqrt(2 * log (총 시도 횟수) / 해당 action이 선택된 횟수)
      ucb_values = self.q_values + np.sqrt((2 * np.log(self.total_counts)) / self.action_counts)
      return np.argmax(ucb_values) # 가장 높은 ucb 값을 가지는 것을 선택

  def update(self, chosen_action, reward):
    self.action_counts[chosen_action] += 1
    self.total_counts += 1
    n = self.action_counts[chosen_action]
    value = self.q_values[chosen_action]

    # 새로운 보상 값 계산
    print(n, value, reward, (1 / float(n)) * reward)
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.q_values[chosen_action] = new_value


In [56]:
# 예제 사용
n_actions = 5
bandit = UCBBandit(n_actions)

# 시뮬레이션
n_simulations = 1000
true_rewards = [0.1, 0.5, 0.8, 0.4, 0.9]
rewards = np.zeros(n_simulations)
from tqdm import tqdm
for i in tqdm(range(n_simulations)):
    chosen_action = bandit.select_action() # 액션 1가지 선택
    reward = np.random.rand() < true_rewards[chosen_action] # 보상 확인
    bandit.update(chosen_action, reward) # 보상에 대한 결과 저장
    rewards[i] = reward

print('각 액션의 예상 보상', bandit.q_values)
print('총 예상 보상', np.sum(rewards))
# 보상이 높은 액션을 더 자주 선택하게 되는 결과 발생

  6%|▌         | 58/1000 [00:00<00:01, 515.08it/s]

[0. 0. 0. 0. 0.]
1.0 0.0 True 1.0
[1. 0. 0. 0. 0.]
1.0 0.0 False 0.0
[1. 0. 0. 0. 0.]
1.0 0.0 True 1.0
[1. 0. 1. 0. 0.]
1.0 0.0 False 0.0
[1. 0. 1. 0. 0.]
1.0 0.0 True 1.0
2.0 1.0 False 0.0
2.0 1.0 True 0.5
2.0 1.0 True 0.5
3.0 1.0 True 0.3333333333333333
3.0 1.0 True 0.3333333333333333
4.0 1.0 True 0.25
4.0 1.0 True 0.25
2.0 0.0 False 0.0
2.0 0.0 False 0.0
5.0 1.0 True 0.2
5.0 1.0 True 0.2
3.0 0.5 False 0.0
6.0 1.0 False 0.0
6.0 1.0 True 0.16666666666666666
7.0 1.0 True 0.14285714285714285
8.0 1.0 True 0.125
9.0 1.0 True 0.1111111111111111
7.0 0.8333333333333334 True 0.14285714285714285
10.0 1.0 True 0.1
8.0 0.8571428571428572 True 0.125
11.0 1.0 True 0.09090909090909091
4.0 0.3333333333333333 True 0.25
3.0 0.0 True 0.3333333333333333
3.0 0.0 True 0.3333333333333333
4.0 0.3333333333333333 False 0.0
4.0 0.3333333333333333 False 0.0
5.0 0.5 True 0.2
9.0 0.875 True 0.1111111111111111
12.0 1.0 True 0.08333333333333333
6.0 0.6000000000000001 False 0.0
10.0 0.8888888888888888 True 0.1
13.0 

 19%|█▉        | 190/1000 [00:00<00:01, 568.64it/s]

 True 0.025
41.0 0.8749999999999996 True 0.024390243902439025
41.0 0.8749999999999996 True 0.024390243902439025
42.0 0.8780487804878044 True 0.023809523809523808
10.0 0.33333333333333337 False 0.0
42.0 0.8780487804878044 True 0.023809523809523808
43.0 0.8809523809523805 True 0.023255813953488372
43.0 0.8809523809523805 True 0.023255813953488372
44.0 0.8837209302325577 False 0.0
44.0 0.8837209302325577 True 0.022727272727272728
45.0 0.8863636363636359 True 0.022222222222222223
46.0 0.8888888888888884 True 0.021739130434782608
47.0 0.8913043478260865 True 0.02127659574468085
9.0 0.25 False 0.0
48.0 0.8936170212765953 True 0.020833333333333332
49.0 0.8958333333333328 True 0.02040816326530612
50.0 0.8979591836734688 True 0.02
51.0 0.8999999999999995 True 0.0196078431372549
52.0 0.901960784313725 True 0.019230769230769232
53.0 0.9038461538461533 True 0.018867924528301886
54.0 0.90566037735849 True 0.018518518518518517
45.0 0.8636363636363632 False 0.0
55.0 0.9074074074074069 True 0.01818181

 33%|███▎      | 329/1000 [00:00<00:01, 611.34it/s]

...

350.0 0.9111747851002863 True 0.002857142857142857
351.0 0.9114285714285711 True 0.002849002849002849
352.0 0.9116809116809114 True 0.002840909090909091
353.0 0.9119318181818179 True 0.0028328611898017
354.0 0.912181303116147 True 0.002824858757062147
355.0 0.9124293785310731 True 0.0028169014084507044
356.0 0.9126760563380278 True 0.0028089887640449437
357.0 0.9129213483146064 True 0.0028011204481792717
358.0 0.9131652661064422 True 0.002793296089385475
359.0 0.91340782122905 True 0.002785515320334262
360.0 0.9136490250696375 True 0.002777777777777778
361.0 0.9138888888888885 True 0.002770083102493075
362.0 0.9141274238227143 False 0.0
363.0 0.9116022099447509 True 0.0027548209366391185
364.0 0.9118457300275478 True 0.0027472527472527475
365.0 0.9120879120879116 True 0.0027397260273972603
366.0 0.9123287671232873 True 0.00273224043715847
367.0 0.9125683060109285 False 0.0
201.0 0.8449999999999991 True 0.004975124378109453
202.0 0.845771144278606 True 0.0049504950495049506
203.0 0.8465

 79%|███████▉  | 793/1000 [00:01<00:00, 567.81it/s]

0.004347826086956522
231.0 0.8434782608695646 True 0.004329004329004329
232.0 0.8441558441558435 True 0.004310344827586207
233.0 0.8448275862068959 True 0.004291845493562232
234.0 0.845493562231759 True 0.004273504273504274
235.0 0.8461538461538455 True 0.00425531914893617
236.0 0.8468085106382972 False 0.0
437.0 0.9082568807339443 False 0.0
33.0 0.4375 False 0.0
438.0 0.9061784897025165 True 0.00228310502283105
439.0 0.9063926940639262 False 0.0
237.0 0.8432203389830502 True 0.004219409282700422
238.0 0.8438818565400837 False 0.0
19.0 0.22222222222222227 False 0.0
19.0 0.22222222222222227 True 0.05263157894736842
20.0 0.26315789473684215 False 0.0
440.0 0.9043280182232338 True 0.0022727272727272726
441.0 0.9045454545454538 True 0.0022675736961451248
442.0 0.904761904761904 False 0.0
239.0 0.8403361344537809 False 0.0
443.0 0.9027149321266961 True 0.002257336343115124
444.0 0.9029345372460489 True 0.0022522522522522522
445.0 0.9031531531531524 True 0.0022471910112359553
446.0 0.9033707

100%|██████████| 1000/1000 [00:01<00:00, 621.83it/s]

 0.0019342359767891683
518.0 0.9110251450676973 False 0.0
21.0 0.25 False 0.0
519.0 0.9092664092664083 True 0.0019267822736030828
520.0 0.9094412331406542 True 0.0019230769230769232
521.0 0.9096153846153837 False 0.0
256.0 0.8392156862745092 True 0.00390625
257.0 0.8398437499999993 True 0.0038910505836575876
258.0 0.8404669260700383 True 0.003875968992248062
259.0 0.8410852713178288 True 0.003861003861003861
260.0 0.8416988416988411 True 0.0038461538461538464
261.0 0.8423076923076918 False 0.0
522.0 0.907869481765834 False 0.0
523.0 0.9061302681992328 True 0.0019120458891013384
524.0 0.9063097514340335 True 0.0019083969465648854
525.0 0.9064885496183197 True 0.0019047619047619048
526.0 0.9066666666666657 True 0.0019011406844106464
527.0 0.9068441064638774 True 0.0018975332068311196
528.0 0.9070208728652742 True 0.001893939393939394
529.0 0.9071969696969688 True 0.001890359168241966
530.0 0.9073724007561428 True 0.0018867924528301887
531.0 0.9075471698113199 True 0.0018832391713747645
5

## 3. Thompson Sampling
- 각 액션에 대한 성공 확률의 분포를 유지하고, 액션을 선택할 때 이 분포를 샘플링하여 선택

- 단계:

  1. 분포 추정 : 각 액션의 보상 확률을 추정하기 위해 베이지안 분포를 사용
  2. 샘플링 : 각 액션에 대한 성공 확률 분포에서 샘플을 추출
  3. 선택 : 샘플링된 성공 확률 중 가장 높은 값을 가지는 액션을 선택
  4. 업데이트 : 선택된 팔의 보상결과를 바탕으로 분포를 업데이트

- 베이지안 접근법이 반영된  Beta 분포 : [0, 1] 범위의 확률을 모델링하는 데 적합

  - Beta(α,β):

    α: 성공 횟수 + 1

    β: 실패 횟수 + 1

  1.초기화 : alpha =1, beta = 1 ->  Beta(1, 1) 분포
  
  모든 성공 확률이 동일하게 가능하다고 가정하는 것을 의미함

  2.업데이트 : 보상 결과에 따라 업데이트
  
  성공하면 alpha / 실패하면 beta 를 증가
  

### 톰슨샘플링과 같은 기법이 '베이지안 접근법' 이라고 불리는 이유?
- 베이지안 통계학의 원리를 사용해 의사결정을 수행하기 때문
  - 데이터와 사전 지식을 결합하여 확률 분포를 업데이트하고 이를 바탕으로 의사결정을 내리는 방법론

1. 사전분포 prior distribution
  - 베이지안 접근법에서는 문제에 대한 초기 가정을 활용해 '사전 분포' 생성가능. 데이터 수집 전의 확률을 제공함

2. 우도함수 likelihood function
  - 주어진 데이터가 특정 확률 모델에 의해 관찰될 확률을 나타냄. 실제 데이터를 기반으로 모델을 업데이트 하는데 사용. 성공과 실패의 횟수를 관찰하여, 해당 데이터를 기반으로 성공 확률의 분포를 업데이트 함

3. 사후 분포 posterior distribution
  - 베이지안 접근법의 핵심은 사후 분포를 계산하는 것. 사전 분포와 우도 함수를 결합하여 주어진 데이터를 기반으로 업데이트된 확률분포를 제공

### Thomson Sampling 에서의 베이지안 접근법

1. 사전 분포 설정 : 각 액션의 성공확률을 Beta 분포로 모델링. 초기에는 Beta(1, 1) 분포를 사용하여 모든 성공 확률이 동등하게 가능하다고 가정

2. 샘플링 : 각 액션의 성공 확률 분포에서 샘플을 추출. 이 샘플링은 각 액션의 성공 확률에 대한 불확실성을 반영

3. 액션 선택 : 샘플링된 성공 확률 증 가장 높은 값을 가지는 팔을 선택. 이 과정은 베이지안 방법론을 통해 얻어진 분포를 기반으로 결정

4. 업데이트 : 실제 보상 결과에 따라 각 액션의 beta 분포 파라미터를 업데이트. 성공하면 alpha, 실패하면 beta 증가

In [57]:
import numpy as np
from scipy.stats import beta


In [58]:
class ThomsonSamplingBandit:
  def __init__(self, n_actions):
    self.n_actions = n_actions
    self.successes = np.zeros(n_actions) # 각 팔의 성공횟수
    self.failures = np.zeros(n_actions) # 각 팔의 실패횟수

  def select_action(self):
    sampled_probs = []
    for action in range(self.n_actions):
      # beta 분포에서 샘플링
      # 성공횟수와 실패횟수에 각각 1씩 더해서 분포에 넣고 확률얻기
      sampled_prob = beta.rvs(self.successes[action]+ 1,
                                self.failures[action] + 1)
      sampled_probs.append(sampled_prob)
    return np.argmax(sampled_probs)

  def update(self, chosen_action, reward):
    if reward == 1 :
      self.successes[chosen_action] += 1
    else:
      self.failures[chosen_action] += 1

In [59]:
n_actions = 5
bandit = ThomsonSamplingBandit(n_actions)

In [62]:
# 시뮬레이션
n_simulations = 1000
true_rewards = [0.1, 0.5, 0.8, 0.4, 0.9]
rewards = np.zeros(n_simulations)

for i in range(n_simulations):
    chosen_action = bandit.select_action() # 액션 중 하나를 고름
    reward = np.random.rand() < true_rewards[chosen_action]
    bandit.update(chosen_action, reward)
    rewards[i] = reward

print(" 각 액션의 성공횟수: ", bandit.successes.astype(int))
print('각 액션의 실패 횟수 : ', bandit.failures)
print('총 보상:', np.sum(rewards))

 각 액션의 성공횟수:  [   0   12   26    1 2679]
각 액션의 실패 횟수 :  [  3.   6.   9.   4. 260.]
총 보상: 928.0
